# Training Classifiers for 6N vs Repaired

In this notebook, we train various classifiers to distinguish between 6N and Repaired categories. The models utilize the hyperparameters optimized in the `_optimisation.ipynb` notebook. The classifiers include Random Forest, SVC, XGBoost, Rocket, MLP, and HIVECOTEV2. We also calculate SHAP values for interpretability, where feasible.

In [ ]:
%pylab
%matplotlib inline
%reload_ext autoreload

# Import necessary libraries
import pandas as pd
import sys
import seaborn as sns
sys.path.append('../src')
import abrTools as at
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report
from collections import Counter
import pretty_confusion_matrix as pcm
from scipy.signal import savgol_filter

# Set the acquisition sampling rate
fs = 195000.0 / 2.0

# Import the datetime module to get today's date
from datetime import date

# Define the folder to save results
savefolder = os.path.join('..', 'results', str(date.today()))

# Create the folder if it does not exist
if not os.path.exists(savefolder):
    os.makedirs(savefolder)

In [ ]:
savefolder

# Train all models - Currently hyperparameters are optimised for F1 score

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.utils.class_weight import compute_sample_weight
from sktime.classification.kernel_based import RocketClassifier
from sktime.classification.hybrid import HIVECOTEV2
from sklearn.feature_selection import f_classif,mutual_info_classif, SelectFpr, SelectPercentile
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from joblib import dump, load

In [ ]:
# Define the frequencies and suffices for the datasets
frequencies = [[100],[100,3000,6000,12000,18000,24000,30000,36000,42000]]
suffices =  ['Click','Global']
results = []
njobs = -1
anovaPercentile= 10

# Loop through each frequency set
for i, freq in enumerate(frequencies):
    print(freq)
    
    # Create classification dataset
    X_train, X_test, y_train, y_test, dataVersion = at.createClassificationDataset(test_size=0.25, oversample=False, ages=[1,], frequencies=freq)
    X = np.vstack([X_train, X_test])
    y = np.hstack([y_train, y_test])

    # Random Forest Classifier with ANOVA feature selection
    print('Forest - anova')
    anova_fs = SelectPercentile(f_classif, percentile=anovaPercentile)
    forest_cl = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', n_jobs=-1, max_depth=None, max_features='sqrt', min_samples_leaf=1, min_samples_split=5, bootstrap=True)
    forest_pip = make_pipeline(anova_fs, forest_cl)
    
    # Check if results already exist, if not, fit the model and save results
    if not os.path.exists(os.path.join(savefolder, f'forest_kFoldCrossValidation_AnovaFS{anovaPercentile}percent'+suffices[i]+'.csv')):
        res = at.fitClassificationModel(forest_pip, X_train, y_train, X_test=X_test, y_test=y_test, saveToWandb=False, modelName='Forest classifier', dataVersion=dataVersion, crossValidation=True, makePlot=False, calculatePValue=False, njobs=njobs, n_splits=5)
        pd.DataFrame(res).to_csv(os.path.join(savefolder, f'forest_kFoldCrossValidation_AnovaFS{anovaPercentile}percent'+suffices[i]+'.csv'))
        results.append(res)
        forest_pip.fit(X_train, y_train)
        dump(forest_pip, os.path.join(savefolder, f'forest_kFoldCrossValidation_AnovaFS{anovaPercentile}percent'+suffices[i]+'.joblib'))
        pd.DataFrame({'y_test': y_test, 'y_predict': forest_pip.predict(X_test)}).to_csv(os.path.join(savefolder, f'forest_kFoldCrossValidation_AnovaFS{anovaPercentile}percent'+suffices[i]+'testResults.csv'))

    # Support Vector Classifier with ANOVA feature selection
    print('svc - anova')
    anova_fs = SelectPercentile(f_classif, percentile=10)
    svc_cl = SVC(probability=True, C=0.01, class_weight='balanced', degree=2, gamma=0.01, kernel='poly', shrinking=True)
    svc_pip = make_pipeline(anova_fs, svc_cl)
    
    # Check if results already exist, if not, fit the model and save results
    if not os.path.exists(os.path.join(savefolder, f'SVC_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.csv')):
        res = at.fitClassificationModel(svc_pip, X_train, y_train, X_test=X_test, y_test=y_test, saveToWandb=False, modelName='SVC classifier', dataVersion=dataVersion, crossValidation=True, makePlot=False, calculatePValue=False, njobs=njobs, n_splits=5)
        pd.DataFrame(res).to_csv(os.path.join(savefolder, f'SVC_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.csv'))
        results.append(res)
        svc_pip.fit(X_train, y_train)
        dump(svc_pip, os.path.join(savefolder, f'SVC_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.joblib'))
        pd.DataFrame({'y_test': y_test, 'y_predict': svc_pip.predict(X_test)}).to_csv(os.path.join(savefolder, f'SVC_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'testResults.csv'))

    # XGBoost Classifier with ANOVA feature selection
    print('xgboost - anova')
    anova_fs = SelectPercentile(f_classif, percentile=10)
    sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)
    reg = xgb.XGBClassifier(n_estimators=200, verbosity=0, n_jobs=-1, random_state=42, max_depth=3, sample_weight=sample_weights, colsample_bytree=0.8, learning_rate=0.1, subsample=0.6)
    xg_pip = make_pipeline(anova_fs, reg)
    
    # Encode labels for XGBoost
    y_train2 = y_train.copy()
    y_train2[y_train=='6N'] = 0
    y_train2[y_train=='Repaired'] = 1
    y_test2 = y_test.copy()
    y_test2[y_test=='6N'] = 0
    y_test2[y_test=='Repaired'] = 1
    y_train2 = y_train2.astype(int)
    y_test2 = y_test2.astype(int)
    
    # Check if results already exist, if not, fit the model and save results
    if not os.path.exists(os.path.join(savefolder, f'XGBOOST_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.csv')):
        res = at.fitClassificationModel(xg_pip, X_train, y_train2, X_test=X_test, y_test=y_test2, saveToWandb=False, modelName='XGBOOST', dataVersion=dataVersion, crossValidation=True, makePlot=False, njobs=njobs, encode_labels=True, n_splits=5)
        pd.DataFrame(res).to_csv(os.path.join(savefolder, f'XGBOOST_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.csv'))
        results.append(res)
        xg_pip.fit(X_train, y_train2)
        dump(xg_pip, os.path.join(savefolder, f'XGBOOST_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.joblib'))
        y_predict2 = xg_pip.predict(X_test)
        y_predict = y_predict2.copy().astype(str)
        y_predict[y_predict2 == 0] = '6N'
        y_predict[y_predict2 == 1] = 'Repaired'
        pd.DataFrame({'y_test': y_test, 'y_predict': y_predict}).to_csv(os.path.join(savefolder, f'XGBOOST_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'testResults.csv'))

    # Rocket Classifier with ANOVA feature selection
    print('rocket - anova')
    anova_fs = SelectPercentile(f_classif, percentile=10)
    rocket = RocketClassifier(num_kernels=5000, n_jobs=-1, max_dilations_per_kernel=16, n_features_per_kernel=2, use_multivariate='yes', random_state=42)
    rocket_pip = make_pipeline(anova_fs, rocket)
    
    # Check if results already exist, if not, fit the model and save results
    if not os.path.exists(os.path.join(savefolder, f'Rocket_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.csv')):
        try:
            res = at.fitClassificationModel(rocket_pip, X_train, y_train, X_test=X_test, y_test=y_test, saveToWandb=False, modelName='Rocket classifier', dataVersion=dataVersion, crossValidation=True, makePlot=False, calculatePValue=False, njobs=njobs, n_splits=5)
        except ValueError:
            res = {'accuracy': array([0, 0, 0]), 'roc_auc_score': array([0, 0, 0])}
        pd.DataFrame(res).to_csv(os.path.join(savefolder, f'Rocket_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.csv'))
        results.append(res)
        rocket_pip.fit(X_train, y_train)
        dump(rocket_pip, os.path.join(savefolder, f'Rocket_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.joblib'))
        pd.DataFrame({'y_test': y_test, 'y_predict': rocket_pip.predict(X_test)}).to_csv(os.path.join(savefolder, f'Rocket_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'testResults.csv'))

    # MLP Classifier with ANOVA feature selection
    print('MLP - anova')
    anova_fs = SelectPercentile(f_classif, percentile=10)
    mlp = MLPClassifier(solver='lbfgs', random_state=42, early_stopping=True, activation='tanh', alpha=0.05, hidden_layer_sizes=(150,), learning_rate_init=0.001, max_iter=100)
    mlp_pip = make_pipeline(anova_fs, mlp)
    
    # Check if results already exist, if not, fit the model and save results
    if not os.path.exists(os.path.join(savefolder, f'MLP_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.csv')):
        res = at.fitClassificationModel(mlp_pip, X_train, y_train, X_test=X_test, y_test=y_test, saveToWandb=False, modelName='Forest classifier', dataVersion=dataVersion, crossValidation=True, makePlot=False, calculatePValue=False, njobs=njobs, n_splits=5)
        pd.DataFrame(res).to_csv(os.path.join(savefolder, f'MLP_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.csv'))
        results.append(res)
        mlp_pip.fit(X_train, y_train)
        dump(mlp_pip, os.path.join(savefolder, f'MLP_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.joblib'))
        pd.DataFrame({'y_test': y_test, 'y_predict': mlp_pip.predict(X_test)}).to_csv(os.path.join(savefolder, f'MLP_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'testResults.csv'))

    # HIVE-COTE Classifier with ANOVA feature selection
    print('hive cote - anova')
    anova_fs = SelectPercentile(f_classif, percentile=10)
    hc2 = HIVECOTEV2(n_jobs=-1, time_limit_in_minutes=0.2, random_state=42)
    hc2_pip = make_pipeline(anova_fs, hc2)
    
    # Check if results already exist, if not, fit the model and save results
    if not os.path.exists(os.path.join(savefolder, f'hivecote_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.csv')):
        try:
            res = at.fitClassificationModel(hc2_pip, X_train, y_train, X_test=X_test, y_test=y_test, saveToWandb=False, modelName='Forest classifier', dataVersion=dataVersion, crossValidation=True, makePlot=False, calculatePValue=False, njobs=njobs, n_splits=5)
        except ValueError:
            res = {'accuracy': array([0, 0, 0]), 'roc_auc_score': array([0, 0, 0])}
        pd.DataFrame(res).to_csv(os.path.join(savefolder, f'hivecote_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.csv'))
        results.append(res)
        hc2_pip.fit(X_train, y_train)
        dump(hc2_pip, os.path.join(savefolder, f'hivecote_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.joblib'))
        pd.DataFrame({'y_test': y_test, 'y_predict': hc2_pip.predict(X_test)}).to_csv(os.path.join(savefolder, f'hivecote_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'testResults.csv'))


# Calculate Shapley values for all models

This calculation is not computationally possible for bigger, non-tree based models (HC, ROCKET and MLP and SVC) as it requires too much RAM. SVC is doable for click datasets

In [ ]:
# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.utils.class_weight import compute_sample_weight
from sktime.classification.kernel_based import RocketClassifier
from sktime.classification.hybrid import HIVECOTEV2
from sklearn.feature_selection import f_classif, mutual_info_classif, SelectFpr, SelectPercentile
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from joblib import dump, load
import shap

# Define the folder to save results
savefolder = '../results/2024-10-25-main-optimisedForF1Score'

# Define the frequencies and suffices for the datasets
frequencies = [[100], [100, 3000, 6000, 12000, 18000, 24000, 30000, 36000, 42000]]
suffices = ['Click', 'Global']
results = []
njobs = -1

# Loop through each frequency set
for i, freq in enumerate(frequencies):
    print(freq)
    
    # Create classification dataset
    X_train, X_test, y_train, y_test, dataVersion = at.createClassificationDataset(test_size=0.25, oversample=False, ages=[1,], frequencies=freq)
    X = np.vstack([X_train, X_test])
    y = np.hstack([y_train, y_test])

    # Loop through each ANOVA percentile
    for anovaPercentile in [10]:  # Modify to use other ANOVA percentiles
        # We use TreeExplainer for RandomForest and XGBoost, and KernelExplainer for all other models

        # Random Forest
        savefile = os.path.join(savefolder, f'forest_kFoldCrossValidation_AnovaFS{anovaPercentile}percent'+suffices[i]+'ShapCoeff.csv')
        if not os.path.exists(savefile):
            try:
                # Load the pre-trained RandomForest model
                model = load(os.path.join(savefolder, f'forest_kFoldCrossValidation_AnovaFS{anovaPercentile}percent'+suffices[i]+'.joblib'))
                
                # Transform the test and train datasets using the feature selector
                X_test_reduced = model[0].transform(X_test)
                X_train_reduced = model[0].transform(X_train)
                
                # Sample 100 instances from the training set for SHAP value calculation
                X100 = shap.utils.sample(X_train_reduced, 100)
                
                # Initialize the SHAP TreeExplainer with the RandomForest model
                ke = shap.TreeExplainer(model[1], X100, approximate=True)
                
                # Calculate SHAP values for the test set
                sv = ke.shap_values(X_test_reduced)
                
                # Inverse transform the SHAP values to the original feature space
                class1Coeff = model[0].inverse_transform(sv[:, :, 0])
                class2Coeff = model[0].inverse_transform(sv[:, :, 1])
                
                # Save the SHAP values to a CSV file
                pd.DataFrame(class1Coeff.T).to_csv(savefile)
            except FileNotFoundError:
                # Handle the case where the model file is not found
                pass

        # XGBoost
        savefile = os.path.join(savefolder, f'XGBOOST_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'ShapCoeff.csv')
        if not os.path.exists(savefile):
            try:
                #Same as above but for XGBoost.
                model = load(os.path.join(savefolder, f'XGBOOST_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.joblib'))
                X_test_reduced = model[0].transform(X_test)
                X_train_reduced = model[0].transform(X_train)
                X100 = shap.utils.sample(X_train_reduced, 100)
                ke = shap.TreeExplainer(model[1], X100, approximate=True)
                sv = ke.shap_values(X_test_reduced)
                class1Coeff = model[0].inverse_transform(sv)
                pd.DataFrame(class1Coeff.T).to_csv(savefile)
            except FileNotFoundError:
                pass

        # Loop through other models 
        for modelname in ['SVC']:  # SVC requires too much memory for the global model (600 GB at the moment)
            print(modelname)
            savefile = os.path.join(savefolder, f'{modelname}_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'ShapCoeff.csv')
            if not os.path.exists(savefile):
                try:
                    model = load(os.path.join(savefolder, f'{modelname}_kFoldCrossValidation_AnovaFS{anovaPercentile}percent_'+suffices[i]+'.joblib'))
                    X_test_reduced = model[0].transform(X_test[:5, :])  # We test on a subsample of data
                    X_train_reduced = model[0].transform(X_train)
                    X100 = shap.utils.sample(X_train_reduced, 50)  # We subsample 50 samples to speed up
                    ke = shap.KernelExplainer(model[1].predict_proba, X100, approximate=True, check_additivity=False) # We use KernelExplainer for SVC
                    sv = ke.shap_values(X_test_reduced)
                    if sv.ndim == 3:
                        class1Coeff = model[0].inverse_transform(sv[:, :, 0])
                    else:
                        if len(sv) > 1:
                            class1Coeff = model[0].inverse_transform(sv[0])
                        else:
                            class1Coeff = model[0].inverse_transform(sv)
                    pd.DataFrame(class1Coeff.T).to_csv(savefile)
                except FileNotFoundError:
                    pass
